In [62]:
import sys
sys.path.append("/home/sara/lib")

In [63]:
import pandas as pd
import os
import numpy
import libopf_py
import pandas as pd
import time

from sklearn.svm import SVC
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [64]:
df = pd.read_csv("csv_dataset/dataframe.csv", header=None, sep=';')
df = df.drop([0], axis=0)
df = df.drop(columns=[0])
features = df.iloc[1:2975,1:137]
features = features.to_numpy()
features = features.astype('float64')
labels = df.iloc[1:,136]
labels = labels.to_numpy()
labels = labels.astype('float64')
n_samples = len(features)

label_train_32 = train_labels.astype(numpy.int32)
label_test_32 = test_labels.astype(numpy.int32)

In [65]:
# adaptações ao OPF.
def opf():
        
    # OPF only supports 32 bits labels at the moment
    label_train_32 = train_labels.astype(numpy.int32)
    label_test_32 = test_labels.astype(numpy.int32)

    O = libopf_py.OPF()

    O.fit(train_feat, label_train_32)

    predicted = O.predict(test_feat)
    name = "OPF"
    acc = accuracy_score(label_test_32, predicted)
    print ("{:46} {:.3f} ".format(name, acc.mean()))

In [66]:
train_feat,test_feat,train_labels,test_labels = train_test_split(features,labels, test_size=0.20,random_state=42 )

print ("Training set:", train_feat.shape[0], "samples")
print ("Test set:", test_feat.shape[0], "samples")

Training set: 2379 samples
Test set: 595 samples


In [67]:
C=1
models = (SVC(kernel='linear', C=C, gamma='auto'),
          SVC(kernel='rbf', gamma=0.5, C=C),
          SVC(kernel='poly', degree=3, C=C, gamma='auto'),
          KNeighborsClassifier(3),
          DecisionTreeClassifier(max_depth=8),
          RandomForestClassifier(max_depth=8, n_estimators=10, max_features=1),          
          AdaBoostClassifier(),
          GaussianNB(),
         )

names = ["SVC with linear kernel","SVC with RBF kernel","SVC with polynomial (degree 3) kernel",
         "K Nearest Neighbors","Decision Tree", "Random Forest", "AdaBoost","Naive Bayes"]


k-holdout validation

In [68]:
print ("      Classifiers: \t Accuracy:".expandtabs(44))
for name, clf in zip(names, models):    
    clf.fit(train_feat,train_labels) #train each model
    scores = clf.score(test_feat,test_labels) #evaluate each model in the test set
    print ("{:46} {:.3f} ".format(name, scores.mean()))
opf()

      Classifiers:                           Accuracy:
SVC with linear kernel                         0.931 
SVC with RBF kernel                            0.064 
SVC with polynomial (degree 3) kernel          0.926 
K Nearest Neighbors                            0.257 
Decision Tree                                  0.975 
Random Forest                                  0.232 
AdaBoost                                       0.316 
Naive Bayes                                    1.000 
OPF                                            0.281 


k-fold cross validation

In [ ]:
print ("      Classifiers: \t Accuracy (standard deviation)".expandtabs(37))
for name, clf in zip(names, models):    
    scores = cross_val_score(clf,features,labels,cv=10) #train and evaluate each model in each fold    
    print ("{:41} {:.3f} (+/- {:.3f}) ".format(name, scores.mean(), scores.std() * 2))

      Classifiers:                    Accuracy (standard deviation)
SVC with linear kernel                    0.887 (+/- 0.055) 
SVC with RBF kernel                       0.065 (+/- 0.001) 
